In [1]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Sat Oct 21 09:54:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8     7W /  N/A |     99MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [18]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]]) 
    random.shuffle(sequential_data)
    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle (buys)
    random.shuffle (sells)
    # We want to know what is the minimum vlaue of this two lists.
    lower = min(len(buys), len(sells))
    # Upto 30K Buys
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    random.shuffle(sequential_data)

    # We will do X, y for model.fit(X, y)
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]
#preprocess_df (main_df)
train_x, train_y = preprocess_df (main_df)
validation_x, validation_y = preprocess_df (validation_main_df)
print(f"Train Data Size: {len(train_x)}, Validation Data Size: {len(validation_x)}")
print(f"Train Data:")
print(f"  Buys List: {train_y.count(1)}")
print(f"  Don't Buy List: {train_y.count(0)}")
print(f"Validation Data:")
print(f"  Buys List: {validation_y.count(1)}")
print(f"  Don't Buys List: {validation_y.count(0)}")

Train Data Size: 69188, Validation Data Size: 3062
Train Data:
  Buys List: 34594
  Don't Buy List: 34594
Validation Data:
  Buys List: 1531
  Don't Buys List: 1531
